In [1]:
import yaml
from trainer import Trainer
import learners
import pickle

In [2]:
def get_args(path):
    class Namespace:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)
            
        def __repr__(self):
            return str(self.__dict__)
    
    with open(path, 'r') as f:
        args = Namespace(**yaml.safe_load(f))
    return args    

In [ ]:
args = get_args('outputs/cifar-100/10-task/coda-p/args.yaml')

# Train.
# 최대한 trainer.py의 evaluate 함수를 그대로 사용하되, return_logits=True로 설정하여 logits를 반환하도록 수정
seed = 0
metric_keys = ['acc','time',]
save_keys = ['global', 'pt', 'pt-local']

trainer = Trainer(args, seed, metric_keys, save_keys)
self = trainer

self.learner = learners.__dict__[self.learner_type].__dict__[self.learner_name](self.learner_config)

metric_table = {}
metric_table_local = {}

for i in range(self.max_task):

    # increment task id in prompting modules
    if i > 0:
        try:
            if self.learner.model.module.prompt is not None:
                self.learner.model.module.prompt.process_task_count()
        except:
            if self.learner.model.prompt is not None:
                self.learner.model.prompt.process_task_count()

    # load model
    model_save_dir = self.model_top_dir + '/models/repeat-'+str(self.seed+1)+'/task-'+self.task_names[i]+'/'
    self.learner.task_count = i 
    self.learner.add_valid_output_dim(len(self.tasks_logits[i]))
    self.learner.pre_steps()
    self.learner.load_model(model_save_dir)

    # set task id for model (needed for prompting)
    try:
        self.learner.model.module.task_id = i
    except:
        self.learner.model.task_id = i

    self.reset_cluster_labels = True
    for j in range(i+1):
        if not i in metric_table:
            metric_table[i] = {}
        metric_table[i][j] = self.task_eval(j, return_logits=True)

    for j in range(i+1):
        if not i in metric_table_local:
            metric_table_local[i] = {}
        metric_table_local[i][j] = self.task_eval(j, local=True, return_logits=True)


In [4]:
with open('metric_table.pkl', 'wb') as f:
    pickle.dump(metric_table, f)
    
with open('metric_table_local.pkl', 'wb') as f:
    pickle.dump(metric_table_local, f)